#  Indexing and searching image based documents (using ColPali with Qdrant)

We can retrieve documents with images such as user guides or old scanned documents. We will use an embedding model for the documents and the queries that supports images. We will also tune the vector database to efficiently store and search these embedding vectors.

Here are the steps:
* [Creating image collection index](#creating-image-collection-index)
* [Searching the image index](#searching-the-image-index)
* [Generating a reply based on the retrieved image](#generate-response-with-the-retrieved-images)

## Visual Improvements

In [ ]:
!pwd
%cd advanced-rag

In [ ]:
# Before installing other requirements, ensure compatible torch and torchvision versions.
# The error indicates torchvision 0.24.0 requires torch 2.9.0, while current torch is 2.4.1.
# We will uninstall existing torch, torchvision, and torchaudio and then install the required compatible versions.
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.9.0 torchvision==0.24.0+cu121 torchaudio==2.9.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html

!git clone https://github.com/guyernest/advanced-rag.git
%cd advanced-rag
!pip install -q -r requirements.txt

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

## Enhanced Visualisation

In [ ]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [ ]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Creating image collection index

### Converting PDF files into images

We don't want to rely on text extraction from the PDF files, and we want to focus on the visual aspects of the pages.

In [ ]:
import os
from pdf2image.pdf2image import convert_from_path, PDFPageCountError

def convert_pdfs_to_images(pdf_folder):
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    all_images = {}
    skipped_pdfs = []

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        try:
            images = convert_from_path(pdf_path)
            all_images[pdf_file] = images
        except PDFPageCountError as e:
            print(f"Skipping '{pdf_file}' due to PDFPageCountError: {e}")
            skipped_pdfs.append(pdf_file)
        except Exception as e:
            print(f"Skipping '{pdf_file}' due to an unexpected error: {e}")
            skipped_pdfs.append(pdf_file)

    if skipped_pdfs:
        print("\nSummary of skipped PDF files:")
        for s_pdf in skipped_pdfs:
            print(f"- {s_pdf}")
    else:
        print("No PDF files were skipped.")

    return all_images

In [ ]:
# all_images = convert_pdfs_to_images("data/ikea/")
# Install poppler-utils
!apt-get install poppler-utils

all_images = convert_pdfs_to_images("/content/advanced-rag/data/pdf_files/tmp")

In [ ]:
console.print(all_images)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,2 , figsize=(15, 10))

first_pdf_key = next(iter(all_images))
for i, ax in enumerate(axes.flat):
    img = all_images[first_pdf_key][i]
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
from colpali_engine.models import ColPali, ColPaliProcessor
import torch


# Initialize ColPali model and processor
model_name = (
    "vidore/colpali-v1.2"  # Use the latest version available
)
colpali_model = ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cpu",  # Use "cuda:0" for GPU, "cpu" for CPU, or "mps" for Apple Silicon
)
colpali_processor = ColPaliProcessor.from_pretrained(
    "vidore/colpaligemma-3b-pt-448-base"
)

In [ ]:
# Fix for 'MessageFactory' AttributeError: Reinstall protobuf
!pip uninstall protobuf -y
!pip install protobuf

In [ ]:
console.print(colpali_model)

In [ ]:
sample_image = all_images[first_pdf_key][0]
with torch.no_grad():
    sample_batch = colpali_processor.process_images([sample_image]).to(
        colpali_model.device
    )
    sample_embedding = colpali_model(**sample_batch)

In [ ]:
console.print(sample_embedding)

In [ ]:
from rich.table import Table

table = Table(title="Document Embedding")
table.add_column("Documents", style="cyan", no_wrap=True)
table.add_column("Tokens", style="bright_yellow")
table.add_column("Vector Size", style="green")

table.add_row(
    str(sample_embedding.shape[0]),
    str(sample_embedding.shape[1]),
    str(sample_embedding.shape[2])
)

console.print(table)

## Store embeddings in a persistent Store

In [ ]:
# Install ChromaDB
import os
!pip install chromadb
import chromadb

# Setup Persistent Storage

persistent_store = '/content/advanced-rag/data/persistent_store'

# Ensure the destination directory exists
if not os.path.exists(persistent_store):
    os.makedirs(persistent_store) # Creates parent directories if needed
%cd '/content/advanced-rag/data/persistent_store'

client = chromadb.PersistentClient(path="./chroma_db")

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
   # ":memory:"
    path="/content/advanced-rag/data/persistent_store"
)  # Use ":memory:" for in-memory database or "path/to/db" for persistent storage

In [ ]:
vector_size = sample_embedding.shape[2]

In [ ]:
from qdrant_client.http import models

multi_vector_params = models.VectorParams(
    size=vector_size,
    distance=models.Distance.COSINE,
    multivector_config=models.MultiVectorConfig(
        comparator=models.MultiVectorComparator.MAX_SIM
    ),
)

### Reducing vector memory using Quantization

We can define a `ScalarQuantizationConfig` and pass it when creating the collection. On the server side, Qdrant will convert the vectors to 8-bit integers, reducing the memory footprint and speeding up the search process. You can also switch on or off the `always_ram` parameter, keeping the vectors in RAM. This will increase performance at the cost of memory usage.

In [ ]:
scalar_quant = models.ScalarQuantizationConfig(
    type=models.ScalarType.INT8,
    quantile=0.99,
    always_ram=False,
)

In [ ]:
collection_name="driver-license"

qdrant_client.recreate_collection(
    collection_name=collection_name,  # the name of the collection
    on_disk_payload=True,  # store the payload on disk
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=100
    ),  # it can be useful to swith this off when doing a bulk upload and then manually trigger the indexing once the upload is done
    vectors_config=models.VectorParams(
        size=vector_size,
        distance=models.Distance.COSINE,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        ),
        quantization_config=models.ScalarQuantization(
            scalar=scalar_quant,
        ),
    ),
)

### Upserting the encoded images into the vector database

We define a helper function to upload points to Qdrant via the client. We use the stamina library to enable retries in case of network issues

In [ ]:
import stamina

@stamina.retry(on=Exception, attempts=3)
def upsert_to_qdrant(batch):
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=points,
            wait=False,
        )
    except Exception as e:
        print(f"Error during upsert: {e}")
        return False
    return True

We will now upload the vectors to qdrant. We do this by creating batches of data, passing it through the ColPali model and then adding the embeddings to a Qdrant `PointStruct`.

In [ ]:
import uuid
from tqdm import tqdm

batch_size = 2  # Adjust based on your GPU memory constraints

total_images = sum(len(images) for images in all_images.values())

# Use tqdm to create a progress bar
with tqdm(total=total_images, desc="Indexing Progress") as pbar:
    for doc_id, pdf_file in enumerate(all_images.keys()):
        for i in range(0, len(all_images[pdf_file]), batch_size):
            images = all_images[pdf_file][i : i + batch_size]

            # Process and encode images
            with torch.no_grad():
                batch_images = colpali_processor.process_images(images).to(
                    colpali_model.device
                )
                image_embeddings = colpali_model(**batch_images)

            # Prepare points for Qdrant
            points = []
            for j, embedding in enumerate(image_embeddings):
                unique_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{doc_id}.{i + j}"))
                # Convert the embedding to a list of vectors
                multivector = embedding.cpu().float().numpy().tolist()
                points.append(
                    models.PointStruct(
                        id=unique_id,
                        vector=multivector,  # This is now a list of vectors
                        payload={
                            "doc": pdf_file,
                            "page": i+j+1
                        },  # can also add other metadata/data
                    )
                )
            # Upload points to Qdrant
            try:
                upsert_to_qdrant(points)
            # clown level error handling here 🤡
            except Exception as e:
                print(f"Error during upsert: {e}")
                continue

            # Update the progress bar
            pbar.update(batch_size)

print("Indexing complete!")

If you had the indexing off during the upload you can trigger an index by setting a lower indexing threshold.

In [ ]:
qdrant_client.update_collection(
    collection_name=collection_name,
    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=10),
)

In [ ]:
console.print(
    qdrant_client
    .get_collection(collection_name)
)

In [ ]:
console.print(
    qdrant_client
    .scroll(
        collection_name=collection_name,
        limit=20
    )
)

## Searching the image index

Once we uploaded the encoded images to the vector database, we can query it.

In [ ]:
# query_text = "What is the license number?"
query_text = "What is Savitha's license number?"
with torch.no_grad():
    batch_query = colpali_processor.process_queries([query_text]).to(
        colpali_model.device
    )
    query_embedding = colpali_model(**batch_query)


In [ ]:
console.print(query_embedding.shape)

In [ ]:
# Convert the query embedding to a list of vectors
multivector_query = query_embedding[0].cpu().float().numpy().tolist()

In [ ]:
search_result = qdrant_client.query_points(
    collection_name=collection_name,
    query=multivector_query,
    limit=3,
    timeout=60,
)

In [ ]:
console.print(search_result)

### Show the search results images

We can display the images that were retrieved by the vector search.

In [ ]:
import matplotlib.pyplot as plt

# Extract the top 3 images from the search result for display
top_images = search_result.points[:6]

# Create a figure with subplots for each image
fig, axs = plt.subplots(1, 3, figsize=(15, 10))

# Iterate over the top images and plot each one
for i, point in enumerate(top_images):
    pdf_file = point.payload.get('doc')
    page_num = int(point.payload.get('page')) - 1
    img = all_images[pdf_file][page_num]
    axs[i].imshow(img)
    axs[i].set_title(f"Score: {point.score}, \n Doc: {pdf_file}")
    axs[i].axis('off')  # Do not display axes for better visualization

plt.tight_layout()
plt.show()

## Generate response with the retrieved image(s)

In the **A**ugmentation step we encode the retrieved image using base64 and send it as part of the prompt to the generation model, alongside the user's query.

In [ ]:
import base64
from io import BytesIO

top_image = search_result.points[0]
pdf_file = top_image.payload.get('doc')
page_num = int(top_image.payload.get('page')) - 1
image = all_images[pdf_file][page_num]
display(image)

buffered = BytesIO()
image.save(buffered, format="PNG")  # You may choose another format if needed
img_bytes = buffered.getvalue()

image1_media_type = "image/png"

image1_data = base64.standard_b64encode(img_bytes).decode("utf-8")

In [ ]:
from google.colab import userdata


# Get the secret value from Colab's secrets manager
openai_key = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

message = client.chat.completions.create(
    model="gpt-4o", # or gpt-4o-mini
   messages=[
        {
            "role": "user",
            "content": [
                {"type": "text",
                 "text": query_text
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image1_data}"
                    },
                },
            ],
        }
    ],
    max_tokens=2000,
)

console.print(message)